In [1]:
import pandas as pd
import numpy as np
import plotly
import plotly.plotly as py
import cufflinks as cf

In [2]:
nl_fac = pd.read_csv(r'C:\Users\David\Python\scrapy\facup\02_merge\nleague_teams_v1.csv')
nl_tab = pd.read_csv(r'C:\Users\David\Python\scrapy\facup\01_league_tables\nonleague_2012-15.csv')

In [4]:
nl_tab = nl_tab[['league','pos','tier','year','team']]

In [6]:
nl_fac = nl_fac[nl_fac.year > 2011]

Text string regularisation

In [10]:
nl_tab['team'] = nl_tab['team'].str.upper()
nl_tab['league'] = nl_tab['league'].str.upper()

In [13]:
#removing 'non-breaking' spaces and normal spaces from end of strings
nl_tab['team'].replace('\xc2\xa0', '', inplace = True, regex = True) 
nl_tab['team'] = nl_tab['team'].str.strip()

In [14]:
nl_tab.head()

,league,pos,tier,year,team
0,NATIONAL,1,501,2012,FLEETWOOD TOWN
1,NATIONAL,2,501,2012,WREXHAM
2,NATIONAL SOUTH,1,602,2012,WOKING
3,NORTHERN PREMIER DIV ONE NORTH,1,801,2012,AFC FYLDE
4,NATIONAL,3,501,2012,MANSFIELD TOWN


#### Merge test 1

In [15]:
nl_merge = pd.merge(nl_tab, nl_fac, how='right', on=['team', 'year'])

missing_teams = nl_merge[nl_merge.pos.isnull()].copy()

mt_plot = missing_teams.year.value_counts().sort_index().reindex()

In [17]:
mt_plot.iplot(kind='bar', dimensions = (700,400))

In [18]:
missing_teams

,league,pos,tier,year,team,round,stage
123,NaN,NaN,NaN,2012,BRADFORD (PARK AVENUE),FIRST ROUND PROPER,8
124,NaN,NaN,NaN,2012,SALISBURY CITY,FIRST ROUND PROPER,8
125,NaN,NaN,NaN,2013,BRADFORD (PARK AVENUE),FIRST ROUND PROPER,8
126,NaN,NaN,NaN,2014,SALISBURY CITY,FIRST ROUND PROPER,8
127,NaN,NaN,NaN,2015,WESTON SUPER MARE,FIRST ROUND PROPER,8


In [19]:
from fuzzywuzzy import process
import fuzzywuzzy
import Levenshtein

In [20]:
def near_match(row):
    return process.extractOne(row['team'], nl_tab[nl_tab.year == row['year']]['team'].values) # only search specific year
     #return tuple containing (nearest match, score)
    
%time missing_teams['match'], missing_teams['score'] = zip(*missing_teams.apply(near_match, axis = 1))

Wall time: 657 ms


In [21]:
missing_teams

,league,pos,tier,year,team,round,stage,match,score
123,NaN,NaN,NaN,2012,BRADFORD (PARK AVENUE),FIRST ROUND PROPER,8,BRADFORD PARK AVENUE,98
124,NaN,NaN,NaN,2012,SALISBURY CITY,FIRST ROUND PROPER,8,SALISBURY,90
125,NaN,NaN,NaN,2013,BRADFORD (PARK AVENUE),FIRST ROUND PROPER,8,BRADFORD PARK AVENUE,98
126,NaN,NaN,NaN,2014,SALISBURY CITY,FIRST ROUND PROPER,8,SALISBURY,90
127,NaN,NaN,NaN,2015,WESTON SUPER MARE,FIRST ROUND PROPER,8,WESTON-SUPER-MARE,100


These matches are correct, will merge will update nl_tab dataframe

Need to update index for update method

In [22]:
nl_tab.set_index(['year', 'team'], inplace=True, drop=False)

In [23]:
nl_tab.head()

league  pos  tier  year  \
year team                                                              
2012 FLEETWOOD TOWN                        NATIONAL    1   501  2012   
     WREXHAM                               NATIONAL    2   501  2012   
     WOKING                          NATIONAL SOUTH    1   602  2012   
     AFC FYLDE       NORTHERN PREMIER DIV ONE NORTH    1   801  2012   
     MANSFIELD TOWN                        NATIONAL    3   501  2012   

                               team  
year team                            
2012 FLEETWOOD TOWN  FLEETWOOD TOWN  
     WREXHAM                WREXHAM  
     WOKING                  WOKING  
     AFC FYLDE            AFC FYLDE  
     MANSFIELD TOWN  MANSFIELD TOWN

In [24]:
missing_teams.set_index(['year', 'match'], inplace=True, drop=False)

In [26]:
missing_teams = missing_teams[['team', 'pos', 'year', 'league', 'tier']]

In [27]:
missing_teams.head()

team  pos  year league  tier
year match                                                               
2012 BRADFORD PARK AVENUE  BRADFORD (PARK AVENUE)  NaN  2012    NaN   NaN
     SALISBURY                     SALISBURY CITY  NaN  2012    NaN   NaN
2013 BRADFORD PARK AVENUE  BRADFORD (PARK AVENUE)  NaN  2013    NaN   NaN
2014 SALISBURY                     SALISBURY CITY  NaN  2014    NaN   NaN
2015 WESTON-SUPER-MARE          WESTON SUPER MARE  NaN  2015    NaN   NaN

In [28]:
nl_tab.update(missing_teams)

#### Merge test 2

In [30]:
nl_merge2 = pd.merge(nl_tab, nl_fac, how='right', on=['team', 'year'])

missing_teams2 = nl_merge2[nl_merge2.pos.isnull()].copy()

mt_plot2 = missing_teams2.year.value_counts().sort_index().reindex()

In [31]:
#making dataframe which join missing teams from 1st and 2nd merge

mt_plot_join = pd.DataFrame({'mt_plot':mt_plot, 'mt_plot2':mt_plot2})

mt_plot_join.iplot(kind='bar', barmode='overlay', dimensions = (700,400))

All teams merged succesfully, now checking for duplicate matches (i.e. teams which exist twice in league tables (per year) and have been matched to fa cup dataframe multiple times 

In [33]:
nl_merge2.groupby(['year', 'team']).filter(lambda x: len(x) > 1)

,league,pos,tier,year,team,round,stage


In [34]:
nl_fac.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 128 entries, 1919 to 2046
Data columns (total 4 columns):
team     128 non-null object
year     128 non-null int64
round    128 non-null object
stage    128 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 5.0+ KB


In [35]:
nl_merge2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 128 entries, 0 to 127
Data columns (total 7 columns):
league    128 non-null object
pos       128 non-null int64
tier      128 non-null int64
year      128 non-null float64
team      128 non-null object
round     128 non-null object
stage     128 non-null float64
dtypes: float64(2), int64(2), object(3)
memory usage: 8.0+ KB


#### Final save of non-league 1952-89 merge

In [36]:
nl_merge2.to_csv(r'C:\Users\David\Python\scrapy\facup\02_merge\fa_nl_merge_2012-15_v1.csv', index=False)